#Primera Vuelta

## Provincias

In [ ]:
import sys
from google.colab import drive
# Conectamos con nuestro Google Drive
drive.mount('/content/drive')
# Establezco una ruta absoluta a un directorio existente de mi Google Drive
BASE_FOLDER = "/content/drive/MyDrive/Colab Notebooks/Vinculacion/"
# agregamos a las paths del sistema la del colab
sys.path.append(BASE_FOLDER)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pandas openpyxl

In [ ]:
import pandas as pd

# Ruta del archivo Excel (.xlsx)
ruta = 'drive/MyDrive/Colab Notebooks/Vinculacion/1996-Data.xlsx'

# Cargar el dataset desde Excel
df = pd.read_excel(ruta)

# Mostrar información básica del dataset
print('Dataset: 1996-Data')
print(f'Número de instancias: {df.shape[0]}')
print(f'Número de atributos: {df.shape[1]}')
print(f'Nombre de los atributos: {df.columns.tolist()}')
print(f'Tipos de datos: \n{df.dtypes}')

Dataset: 1996-Data
Número de instancias: 1117
Número de atributos: 67
Nombre de los atributos: ['AÑO', 'DIGNIDAD', 'REGION', 'PROVINCI', 'CANTON', 'PARROQUIA', 'JUNTAS ', 'ELECTORES', 'PCE1', 'PLRE2', 'PSE3', 'PD3', 'PAB3', 'CFP4', 'ARNE5', 'DP5', 'PSC6', 'PPP7', 'PCD7', 'CID8', 'PR8', 'PUR8', 'UDP9', 'FADI9', 'FNV10', 'PRE10', 'PNR11', 'PLN11', 'AN11', 'ID12', 'APRE13', 'FRA14', 'MPD15', 'PDP16', 'UPL16', 'PSE17', 'PK18', 'UCI19', 'ITI20', 'LISTA21', 'LISTA22', 'LISTA23', 'LISTA24', 'LISTA25', 'LISTA26', 'LISTA27', 'LISTA28', 'LISTA29', 'LISTA30', 'LISTA31', 'LISTA32', 'LISTA33', 'LISTA34', 'LISTA35', 'PLRE - FRA', 'ALIANZA2', 'ALIANZA3', 'ALIANZA4', 'ID -PSFA', 'ALIANZA6', 'VOTOSVAL', 'VOTOSNUL', 'VOTOSBLA', 'VOTOSESC', 'ABSTENCI', 'ACTASVAL', 'VOTOS_CANDIDATOS_1996']
Tipos de datos: 
AÑO                      float64
DIGNIDAD                 float64
REGION                   float64
PROVINCI                 float64
CANTON                   float64
                          ...   
VOTO

In [ ]:
import pandas as pd

# Cargar el archivo Excel
ruta = 'drive/MyDrive/Colab Notebooks/Vinculacion/1996-Data.xlsx'
df = pd.read_excel(ruta)

# Limpiar y filtrar por provincias seleccionadas
df['PROVINCI'] = df['PROVINCI'].astype(str).str.strip().str.replace('.0$', '', regex=True)
provincias_seleccionadas = ['19', '20', '21']
df_filtrado = df[df['PROVINCI'].isin(provincias_seleccionadas)]

# Agrupar por provincia y sumar los votos totales
votos_por_provincia = df_filtrado.groupby('PROVINCI')['VOTOS_CANDIDATOS_1996'].sum().reset_index()

# Renombrar las provincias con sus nombres reales
votos_por_provincia['PROVINCI'] = votos_por_provincia['PROVINCI'].replace({
    '19': 'ZAMORA CHINCHIPE',
    '20': 'GALÁPAGOS',
    '21': 'SUCUMBÍOS'
})

# Renombrar columnas para claridad
votos_por_provincia.columns = ['Provincia', 'Total de Votos']

# Mostrar resultados
print(votos_por_provincia)

          Provincia  Total de Votos
0  ZAMORA CHINCHIPE         18916.0
1         GALÁPAGOS          4001.0
2         SUCUMBÍOS         22093.0


In [ ]:
import pandas as pd

# Cargar el archivo Excel
ruta = 'drive/MyDrive/Colab Notebooks/Vinculacion/1996-Data.xlsx'
df = pd.read_excel(ruta)

# Limpiar y filtrar por provincias seleccionadas (códigos 19, 20, 21)
df['PROVINCI'] = df['PROVINCI'].astype(str).str.strip().str.replace('.0$', '', regex=True)
provincias_seleccionadas = ['19', '20', '21']
df_filtrado = df[df['PROVINCI'].isin(provincias_seleccionadas)]

# Renombrar códigos de provincia por sus nombres reales
df_filtrado['Provincia'] = df_filtrado['PROVINCI'].replace({
    '19': 'ZAMORA CHINCHIPE',
    '20': 'GALÁPAGOS',
    '21': 'SUCUMBÍOS'
})

# Diccionario que mapea a cada candidato con su(s) columna(s) de votos
mapeo_candidatos = {
    'NOBOA RICARDO': ['PLRE - FRA'],
    'PAZ RODRIGO': ['DP5'],
    'NEBOT JAIME': ['PSC6'],
    'BUCARAM ABDALÁ': ['PRE10'],
    'VARGAS FRANK': ['APRE13'],
    'CASTELLÓ JUAN': ['MPD15'],
    'EHLEARS FREDDY': ['PK18'],
    'GALLARDO JOSÉ': ['UCI19'],
    'VELÁZQUEZ JACINTO': ['ITI20']
}

# Calcular votos por provincia y candidato
resultados = []

for candidato, columnas in mapeo_candidatos.items():
    df_votos = df_filtrado.groupby('Provincia')[columnas].sum(numeric_only=True).sum(axis=1).reset_index()
    df_votos['Candidato'] = candidato
    df_votos.rename(columns={0: 'Total_Votos'}, inplace=True)
    resultados.append(df_votos)

# Combinar todos los resultados en un solo DataFrame
df_resultados = pd.concat(resultados, ignore_index=True)

# Reordenar columnas
df_resultados = df_resultados[['Provincia', 'Candidato', 'Total_Votos']]

# Agregar porcentajes por provincia
def calcular_porcentaje(group):
    total = group['Total_Votos'].sum()
    group['Porcentaje'] = (group['Total_Votos'] / total * 100).round(2)
    return group

df_resultados = df_resultados.groupby('Provincia').apply(calcular_porcentaje).reset_index(drop=True)

# Mostrar resultado final
print("\nVotos y porcentaje por candidato y provincia:")
print(df_resultados)

# Opcional: Exportar a Excel
df_resultados.to_excel('Votos_Por_Candidato_Y_Provincia.xlsx', index=False)
print("\nArchivo guardado como 'Votos_Por_Candidato_Y_Provincia.xlsx'")


Votos y porcentaje por candidato y provincia:
           Provincia          Candidato  Total_Votos  Porcentaje
0          GALÁPAGOS      NOBOA RICARDO        107.0        2.67
1          GALÁPAGOS        PAZ RODRIGO        544.0       13.60
2          GALÁPAGOS        NEBOT JAIME       1274.0       31.84
3          GALÁPAGOS     BUCARAM ABDALÁ       1078.0       26.94
4          GALÁPAGOS       VARGAS FRANK        156.0        3.90
5          GALÁPAGOS      CASTELLÓ JUAN         21.0        0.52
6          GALÁPAGOS     EHLEARS FREDDY        788.0       19.70
7          GALÁPAGOS      GALLARDO JOSÉ         24.0        0.60
8          GALÁPAGOS  VELÁZQUEZ JACINTO          9.0        0.22
9          SUCUMBÍOS      NOBOA RICARDO        193.0        0.87
10         SUCUMBÍOS        PAZ RODRIGO       2669.0       12.08
11         SUCUMBÍOS        NEBOT JAIME       3726.0       16.87
12         SUCUMBÍOS     BUCARAM ABDALÁ       4276.0       19.35
13         SUCUMBÍOS       VARGAS FRANK    

/tmp/ipython-input-35-1562946402.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['Provincia'] = df_filtrado['PROVINCI'].replace({
/tmp/ipython-input-35-1562946402.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_resultados = df_resultados.groupby('Provincia').apply(calcular_porcentaje).reset_index(drop=True)


## Cantónes

In [ ]:
import pandas as pd

# Cargar el archivo Excel
ruta = 'drive/MyDrive/Colab Notebooks/Vinculacion/1996-Data.xlsx'
df = pd.read_excel(ruta)

# Limpiar y convertir a string
df['PROVINCI'] = df['PROVINCI'].astype(str).str.strip()
df['CANTON'] = df['CANTON'].astype(str).str.strip()

# Provincias y cantones seleccionados
provincia_guayas = ['9']
cantones_seleccionados = ['425', '430', '435', '505', '510', '675', '715', '735', '790', '820', '835', '870', '890']

# Filtrar por provincia y cantón
df_filtrado = df[df['PROVINCI'].isin(provincia_guayas)]
df_filtrado = df_filtrado[df_filtrado['CANTON'].isin(cantones_seleccionados)]

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# Cargar el archivo Excel
ruta = 'drive/MyDrive/Colab Notebooks/Vinculacion/1996-Data.xlsx'
df = pd.read_excel(ruta)

# Limpiar columnas PROVINCI y CANTON
df['PROVINCI'] = df['PROVINCI'].astype(str).str.strip().str.replace('.0$', '', regex=True)
df['CANTON'] = df['CANTON'].astype(str).str.strip().str.replace('.0$', '', regex=True)

# Provincia seleccionada: Guayas
provincia_guayas = ['9']

# Cantones seleccionados de Guayas
cantones_seleccionados = ['425', '430', '435', '505', '510', '675', '715', '735', '790', '820', '835', '870', '890']

# Filtrar por provincia y cantones
df_filtrado = df[df['PROVINCI'].isin(provincia_guayas)]
df_filtrado = df_filtrado[df_filtrado['CANTON'].isin(cantones_seleccionados)]

# Mapeo de nombres de cantones (opcional, para mostrar nombres reales)
mapeo_cantones = {
    '425': 'SAMBORONDON',
    '430': 'SALITRE',
    '435': 'NARANJAL',
    '505': 'EL TRIUNFO',
    '510': 'NARANJITO',
    '675': 'PALESTINA',
    '715': 'PEDRO CARBO',
    '735': 'PLAYAS',
    '790': 'SIMON BOLIVAR',
    '820': 'DURAN',
    '835': 'A.BAQUERIZO MORENO',
    '870': 'SANTA LUCIA',
    '890': 'CRNL MARCELINO MARIDUENAS'
}

# Reemplazar códigos por nombres de cantones
df_filtrado['Canton_Nombre'] = df_filtrado['CANTON'].map(mapeo_cantones)

# Diccionario que mapea a cada candidato con su(s) columna(s) de votos
mapeo_candidatos = {
    'NOBOA RICARDO': ['PLRE - FRA'],
    'PAZ RODRIGO': ['DP5'],
    'NEBOT JAIME': ['PSC6'],
    'BUCARAM ABDALÁ': ['PRE10'],
    'VARGAS FRANK': ['APRE13'],
    'CASTELLÓ JUAN': ['MPD15'],
    'EHLEARS FREDDY': ['PK18'],
    'GALLARDO JOSÉ': ['UCI19'],
    'VELÁZQUEZ JACINTO': ['ITI20']
}

# Calcular votos por cantón y candidato
resultados = []

for candidato, columnas in mapeo_candidatos.items():
    df_votos = df_filtrado.groupby('Canton_Nombre')[columnas].sum(numeric_only=True).sum(axis=1).reset_index()
    df_votos['Candidato'] = candidato
    df_votos.rename(columns={0: 'Total_Votos'}, inplace=True)
    resultados.append(df_votos)

# Combinar todos los resultados en un solo DataFrame
df_resultados = pd.concat(resultados, ignore_index=True)

# Reordenar columnas
df_resultados = df_resultados[['Canton_Nombre', 'Candidato', 'Total_Votos']]

# Agregar porcentajes por cantón
def calcular_porcentaje(group):
    total = group['Total_Votos'].sum()
    group['Porcentaje (%)'] = (group['Total_Votos'] / total * 100).round(2)
    return group

df_resultados = df_resultados.groupby('Canton_Nombre').apply(calcular_porcentaje).reset_index(drop=True)

# Mostrar resultado final
print("\nVotos y porcentaje por candidato y cantón:")
print(df_resultados)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Votos y porcentaje por candidato y cantón:
          Canton_Nombre          Candidato  Total_Votos  Porcentaje (%)
0    A.BAQUERIZO MORENO      NOBOA RICARDO        108.0            2.01
1    A.BAQUERIZO MORENO        PAZ RODRIGO        481.0            8.95
2    A.BAQUERIZO MORENO        NEBOT JAIME       1335.0           24.85
3    A.BAQUERIZO MORENO     BUCARAM ABDALÁ       3042.0           56.63
4    A.BAQUERIZO MORENO       VARGAS FRANK        165.0            3.07
..                  ...                ...          ...             ...
112       SIMON BOLIVAR       VARGAS FRANK         81.0            1.70
113       SIMON BOLIVAR      CASTELLÓ JUAN         76.0            1.59
114       SIMON BOLIVAR     EHLEARS FREDDY        355.0            7.43
115       SIMON BOLIVAR      GALLARDO JOSÉ         33.0            0.69
116       SIMON BOLIVAR  VELÁZQUEZ 

/tmp/ipython-input-37-3906777402.py:78: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_resultados = df_resultados.groupby('Canton_Nombre').apply(calcular_porcentaje).reset_index(drop=True)


In [ ]:
# Lista de tus 13 cantones seleccionados
cantones_lista = [
    'SAMBORONDON', 'SALITRE', 'NARANJAL', 'EL TRIUNFO', 'NARANJITO',
    'PALESTINA', 'PEDRO CARBO', 'PLAYAS', 'SIMON BOLIVAR', 'DURAN',
    'A.BAQUERIZO MORENO', 'SANTA LUCIA', 'CRNL MARCELINO MARIDUENAS'
]

print("Validación por cantón:\n")

for canton in cantones_lista:
    # Calcular total de votos por candidato (desde df_resultados)
    total_calculado = df_resultados[df_resultados['Canton_Nombre'] == canton]['Total_Votos'].sum()

    # Obtener el valor real del dataset original
    total_real = df_filtrado[df_filtrado['Canton_Nombre'] == canton]['VOTOS_CANDIDATOS_1996'].sum()

    # Mostrar resultados
    print(f"Cantón: {canton}")
    print(f"Total calculado: {int(total_calculado)}")
    print(f"Total real (VOTOS_CANDIDATOS_1996): {int(total_real)}")

    if abs(total_calculado - total_real) < 1:
        print("✅ ¡Coinciden! Los cálculos están alineados con el dataset.")
    else:
        print("⚠️ ¡No coinciden! Revisa posibles errores en las columnas de votos usadas.")

    print("-" * 50)

Validación por cantón:

Cantón: SAMBORONDON
Total calculado: 12209
Total real (VOTOS_CANDIDATOS_1996): 12209
✅ ¡Coinciden! Los cálculos están alineados con el dataset.
--------------------------------------------------
Cantón: SALITRE
Total calculado: 12131
Total real (VOTOS_CANDIDATOS_1996): 12131
✅ ¡Coinciden! Los cálculos están alineados con el dataset.
--------------------------------------------------
Cantón: NARANJAL
Total calculado: 12210
Total real (VOTOS_CANDIDATOS_1996): 12210
✅ ¡Coinciden! Los cálculos están alineados con el dataset.
--------------------------------------------------
Cantón: EL TRIUNFO
Total calculado: 10439
Total real (VOTOS_CANDIDATOS_1996): 10439
✅ ¡Coinciden! Los cálculos están alineados con el dataset.
--------------------------------------------------
Cantón: NARANJITO
Total calculado: 10418
Total real (VOTOS_CANDIDATOS_1996): 10418
✅ ¡Coinciden! Los cálculos están alineados con el dataset.
--------------------------------------------------
Cantón: PAL

### Guardar resultados

In [ ]:
# Opcional: Exportar a Excel
df_resultados.to_excel('Votos_Por_Candidato_Y_Cantón.xlsx', index=False)
print("\nArchivo guardado como 'Votos_Por_Candidato_Y_Cantón.xlsx'")

#df_resultados.to_excel('/content/drive/MyDrive/Votos_Por_Candidato_Y_Cantón.xlsx', index=False)
#print("Archivo guardado en Google Drive")


Archivo guardado como 'Votos_Por_Candidato_Y_Cantón.xlsx'
Archivo guardado en Google Drive


## Parróquias

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# Cargar el archivo Excel
ruta = 'drive/MyDrive/Colab Notebooks/Vinculacion/1996-Data.xlsx'
df = pd.read_excel(ruta)

# Limpiar y preparar columnas
df['PROVINCI'] = df['PROVINCI'].astype(str).str.strip().str.replace('.0$', '', regex=True)
df['PARROQUIA'] = df['PARROQUIA'].astype(str).str.strip().str.replace('.0$', '', regex=True)

# Provincia seleccionada
provincia_seleccionada = ['17']
df_filtrado = df[df['PROVINCI'].isin(provincia_seleccionada)]

# Lista de parroquias en el orden deseado
parroquias_seleccionadas = [
    '30', '80'
]

# Filtrar por parroquias seleccionadas
df_filtrado = df_filtrado[df_filtrado['PARROQUIA'].isin(parroquias_seleccionadas)]

# Mapeo de candidatos y sus columnas de votos
mapeo_candidatos = {
    'NOBOA RICARDO': ['PLRE - FRA'],
    'PAZ RODRIGO': ['DP5'],
    'NEBOT JAIME': ['PSC6'],
    'BUCARAM ABDALÁ': ['PRE10'],
    'VARGAS FRANK': ['APRE13'],
    'CASTELLÓ JUAN': ['MPD15'],
    'EHLEARS FREDDY': ['PK18'],
    'GALLARDO JOSÉ': ['UCI19'],
    'VELÁZQUEZ JACINTO': ['ITI20']
}

# Calcular votos por parroquia y candidato
resultados = []

for candidato, columnas in mapeo_candidatos.items():
    df_votos = df_filtrado.groupby('PARROQUIA')[columnas].sum(numeric_only=True).sum(axis=1).reset_index()
    df_votos['Candidato'] = candidato
    df_votos.rename(columns={0: 'Total_Votos'}, inplace=True)
    resultados.append(df_votos)

# Combinar todos los resultados
df_resultados = pd.concat(resultados, ignore_index=True)

# Calcular porcentaje por parroquia
def calcular_porcentaje(group):
    total = group['Total_Votos'].sum()
    group['Porcentaje (%)'] = (group['Total_Votos'] / total * 100).round(2)
    return group

df_resultados = df_resultados.groupby('PARROQUIA').apply(calcular_porcentaje).reset_index(drop=True)

#Reordenar el DataFrame según el orden personalizado de parroquias_seleccionadas
df_resultados['PARROQUIA'] = pd.Categorical(df_resultados['PARROQUIA'], categories=parroquias_seleccionadas, ordered=True)
df_resultados = df_resultados.sort_values('PARROQUIA').reset_index(drop=True)

# Mostrar los resultados en el orden correcto
print("Votos y porcentaje por candidato y parroquia (en el orden deseado):")
print(df_resultados)

# Exportar a Excel (opcional)
#df_resultados.to_excel('Votos_Por_Parroquia_y_Candidato_Pastaza.xlsx', index=False)
#print("\nArchivo guardado como 'Votos_Por_Parroquia_y_Candidato.xlsx'")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Votos y porcentaje por candidato y parroquia (en el orden deseado):
   PARROQUIA  Total_Votos          Candidato  Porcentaje (%)
0         30         69.0      NOBOA RICARDO            1.76
1         30         44.0      GALLARDO JOSÉ            1.12
2         30       1723.0     EHLEARS FREDDY           43.93
3         30         83.0      CASTELLÓ JUAN            2.12
4         30         35.0  VELÁZQUEZ JACINTO            0.89
5         30        414.0     BUCARAM ABDALÁ           10.56
6         30        341.0        NEBOT JAIME            8.69
7         30        828.0        PAZ RODRIGO           21.11
8         30        385.0       VARGAS FRANK            9.82
9         80         85.0      GALLARDO JOSÉ            1.12
10        80        162.0      NOBOA RICARDO            2.14
11        80        882.0        PAZ RODRIGO           11.67
12        

/tmp/ipython-input-56-4095655767.py:57: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_resultados = df_resultados.groupby('PARROQUIA').apply(calcular_porcentaje).reset_index(drop=True)


In [ ]:
# Sumar los votos de los candidatos por parroquia
votos_candidatos = df_resultados.groupby('PARROQUIA')['Total_Votos'].sum().reset_index()
votos_candidatos.rename(columns={'Total_Votos': 'Votos_Candidatos_Calculados'}, inplace=True)

# Sumar la columna total del dataset por parroquia
votos_reales = df_filtrado.groupby('PARROQUIA')['VOTOS_CANDIDATOS_1996'].sum().reset_index()

# Unir ambas tablas
validacion = pd.merge(votos_candidatos, votos_reales, on='PARROQUIA')

# Calcular si coinciden
validacion['Coincide'] = validacion['Votos_Candidatos_Calculados'] == validacion['VOTOS_CANDIDATOS_1996']

# Mostrar resultados
print("Validación de votos por parroquia:")
print(validacion[['PARROQUIA', 'Votos_Candidatos_Calculados', 'VOTOS_CANDIDATOS_1996', 'Coincide']])

Validación de votos por parroquia:
  PARROQUIA  Votos_Candidatos_Calculados  VOTOS_CANDIDATOS_1996  Coincide
0        30                       3922.0                 3922.0      True
1        80                       7556.0                 7556.0      True


/tmp/ipython-input-57-399190592.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  votos_candidatos = df_resultados.groupby('PARROQUIA')['Total_Votos'].sum().reset_index()


# Segunda Vuelta

## Provincias

In [ ]:
import sys
from google.colab import drive
# Conectamos con nuestro Google Drive
drive.mount('/content/drive')
# Establezco una ruta absoluta a un directorio existente de mi Google Drive
BASE_FOLDER = "/content/drive/MyDrive/Colab Notebooks/Vinculacion/"
# agregamos a las paths del sistema la del colab
sys.path.append(BASE_FOLDER)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pandas openpyxl

In [ ]:
import pandas as pd

# Ruta del archivo Excel (.xlsx)
ruta = 'drive/MyDrive/Colab Notebooks/Vinculacion/COMPILADO 1996 - Actualizado.xlsx'

# Cargar el dataset desde Excel
df = pd.read_excel(ruta)

# Mostrar información básica del dataset
print('Dataset: COMPILADO - 1996')
print(f'Número de instancias: {df.shape[0]}')
print(f'Número de atributos: {df.shape[1]}')
print(f'Nombre de los atributos: {df.columns.tolist()}')
print(f'Tipos de datos: \n{df.dtypes}')

Dataset: COMPILADO - 1996
Número de instancias: 8912
Número de atributos: 66
Nombre de los atributos: ['AÑO', 'DIGNIDAD', 'REGION', 'PROVINCI', 'CANTON', 'PARROQUIA', 'JUNTAS ', 'ELECTORES', 'PCE1', 'PLRE2', 'PSE3', 'PD3', 'PAB3', 'CFP4', 'ARNE5', 'DP5', 'PSC6', 'PPP7', 'PCD7', 'CID8', 'PR8', 'PUR8', 'UDP9', 'FADI9', 'FNV10', 'PRE10', 'PNR11', 'PLN11', 'AN11', 'ID12', 'APRE13', 'FRA14', 'MPD15', 'PDP16', 'UPL16', 'PSE17', 'PK18', 'UCI19', 'ITI20', 'LISTA21', 'LISTA22', 'LISTA23', 'LISTA24', 'LISTA25', 'LISTA26', 'LISTA27', 'LISTA28', 'LISTA29', 'LISTA30', 'LISTA31', 'LISTA32', 'LISTA33', 'LISTA34', 'LISTA35', 'PLRE - FRA', 'ALIANZA2', 'ALIANZA3', 'ALIANZA4', 'ID -PSFA', 'ALIANZA6', 'VOTOSVAL', 'VOTOSNUL', 'VOTOSBLA', 'VOTOSESC', 'ABSTENCI', 'ACTASVAL']
Tipos de datos: 
AÑO          int64
DIGNIDAD    object
REGION      object
PROVINCI    object
CANTON       int64
             ...  
VOTOSNUL     int64
VOTOSBLA     int64
VOTOSESC     int64
ABSTENCI     int64
ACTASVAL     int64
Length: 66,

In [ ]:
import pandas as pd

# Cargar el archivo Excel
ruta = 'drive/MyDrive/Colab Notebooks/Vinculacion/COMPILADO 1996 - Actualizado.xlsx'
df = pd.read_excel(ruta)

# Limpiar y filtrar por **segunda vuelta**
df['DIGNIDAD'] = df['DIGNIDAD'].astype(str).str.strip()
df_segunda_vuelta = df[df['DIGNIDAD'] == 'PRESIDENTE Y VICEPRESIDENTE. SEGUNDA VUELTA']

# Limpiar y filter por provincias seleccionadas (using province names)
df_segunda_vuelta['PROVINCI'] = df_segunda_vuelta['PROVINCI'].astype(str).str.strip()
provincias_seleccionadas = ['PICHINCHA', 'TUNGURAHUA', 'ZAMORA CHINCHIPE']
df_filtrado = df_segunda_vuelta[df_segunda_vuelta['PROVINCI'].isin(provincias_seleccionadas)].copy()

# Mapeo de candidatos y sus columnas de votos
# Note: Only include candidates who were in the second round
mapeo_candidatos = {
    'BUCARAM ABDALÁ': ['PRE10'],
    'NEBOT JAIME': ['PSC6']
}


# Calcular votos por provincia y candidato
resultados = []

for candidato, columnas in mapeo_candidatos.items():
    df_votos = df_filtrado.groupby('PROVINCI')[columnas].sum(numeric_only=True).sum(axis=1).reset_index()
    df_votos['Candidato'] = candidato
    df_votos.rename(columns={0: 'Total_Votos'}, inplace=True)
    resultados.append(df_votos)

# Combinar todos los resultados
df_resultados = pd.concat(resultados, ignore_index=True)

# Reordenar columnas
df_resultados = df_resultados[['PROVINCI', 'Candidato', 'Total_Votos']]

# Agregar porcentajes por provincia
def calcular_porcentaje(group):
    total = group['Total_Votos'].sum()
    group['Porcentaje (%)'] = (group['Total_Votos'] / total * 100).round(2)
    return group

df_resultados = df_resultados.groupby('PROVINCI', group_keys=False).apply(calcular_porcentaje).reset_index(drop=True) # Add group_keys=False

# Renombrar códigos de provincia por sus nombres reales
df_resultados.rename(columns={'PROVINCI': 'Provincia'}, inplace=True)

# Mostrar resultado final
print("\nVotos y porcentaje por candidato y provincia (segunda vuelta):")
print(df_resultados[['Provincia', 'Candidato', 'Total_Votos', 'Porcentaje (%)']])

# Opcional: Exportar a Excel
#df_resultados.to_excel('Resultados_Segunda_Vuelta.xlsx', index=False)
#print("\nArchivo guardado como 'Resultados_Segunda_Vuelta.xlsx'")


Votos y porcentaje por candidato y provincia (segunda vuelta):
          Provincia       Candidato  Total_Votos  Porcentaje (%)
0         PICHINCHA  BUCARAM ABDALÁ       417629           50.31
1        TUNGURAHUA  BUCARAM ABDALÁ        95441           54.91
2  ZAMORA CHINCHIPE  BUCARAM ABDALÁ        15817           75.90
3         PICHINCHA     NEBOT JAIME       412547           49.69
4        TUNGURAHUA     NEBOT JAIME        78378           45.09
5  ZAMORA CHINCHIPE     NEBOT JAIME         5023           24.10


/tmp/ipython-input-43-2606192392.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_segunda_vuelta['PROVINCI'] = df_segunda_vuelta['PROVINCI'].astype(str).str.strip()
/tmp/ipython-input-43-2606192392.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_resultados = df_resultados.groupby('PROVINCI', group_keys=False).apply(calcular_porcentaje).reset_index(drop=True) # Add group_keys=False


In [ ]:
# Agrupar los votos totales del dataset original por provincia
votos_reales = df_segunda_vuelta.groupby('PROVINCI')['VOTOSVAL'].sum().reset_index()
votos_reales.rename(columns={'VOTOSVAL': 'Total_Real'}, inplace=True)

# Agrupar los votos calculados (PRE10 + PSC6) por provincia
votos_calculados = df_resultados.groupby('Provincia')['Total_Votos'].sum().reset_index()
votos_calculados.rename(columns={'Total_Votos': 'Total_Calculado'}, inplace=True)

# Unir ambos resultados
validacion = pd.merge(votos_reales, votos_calculados, left_on='PROVINCI', right_on='Provincia', how='inner')

# Calcular si coinciden
validacion['Coincide'] = validacion['Total_Real'] == validacion['Total_Calculado']

# Mostrar resultados
print("\nValidación de votos por provincia:")
print(validacion[['Provincia', 'Total_Real', 'Total_Calculado', 'Coincide']])


Validación de votos por provincia:
          Provincia  Total_Real  Total_Calculado  Coincide
0         PICHINCHA      830176           830176      True
1        TUNGURAHUA      173819           173819      True
2  ZAMORA CHINCHIPE       20840            20840      True
